In [1]:
import os

os.environ["OPENAI_API_KEY"] = "you_api_key"

In [2]:
%pip install datasets

  Using cached dill-0.3.8-py3-none-any.whl (116 kB)


You should consider upgrading via the 'C:\Users\HP\Documents\2ite2\pfa\model\Question-generation-and-long-form-answers\venv\Scripts\python.exe -m pip install --upgrade pip' command.


# Generate Answers of a Model

## Loading the V2 dataset

In [2]:
from datasets import load_dataset

# loading the V2 dataset
amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")
amnesty_qa

c:\Users\ay0ub\Desktop\PFA\Question generation and long form answers\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the module from C:\Users\ay0ub\.cache\huggingface\modules\datasets_modules\datasets\explodinggradients--amnesty_qa\d0ed9800191a31943ee52a5c22ee4305e28a33f5edcd9a323802112cff07cc24 (last modified on Fri Apr 12 13:16:47 2024) since it couldn't be found locally at explodinggradients/amnesty_qa, or remotely on the Hugging Face Hub.
Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    eval: Dataset({
        features: ['question', 'ground_truth', 'answer', 'contexts'],
        num_rows: 20
    })
})

In [3]:
df = amnesty_qa['eval'].to_pandas()
df

,question,ground_truth,answer,contexts
0,What are the global implications of the USA Su...,The global implications of the USA Supreme Cou...,The global implications of the USA Supreme Cou...,"[- In 2022, the USA Supreme Court handed down ..."
1,Which companies are the main contributors to G...,"According to the Carbon Majors database, the m...","According to the Carbon Majors database, the m...","[- Fossil fuel companies, whether state or pri..."
2,Which private companies in the Americas are th...,The largest private companies in the Americas ...,"According to the Carbon Majors database, the l...",[The private companies responsible for the mos...
3,What action did Amnesty International urge its...,Amnesty International urged its supporters to ...,Amnesty International urged its supporters to ...,[Amnesty International called on its vast netw...
4,What are the recommendations made by Amnesty I...,The recommendations made by Amnesty Internatio...,Amnesty International made several recommendat...,[Amnesty International recommends that the Spe...
5,Who are the target audience of the two books c...,The target audience of the two books created b...,The target audience of the two books created b...,[Amnesty International has therefore created t...
6,Which right guarantees access to comprehensive...,The right that guarantees access to comprehens...,The right that guarantees access to comprehens...,[26. The Act raises serious questions about it...
7,Who has the right to be fully informed about h...,The victims of gross human rights violations a...,Everyone has the right to be fully informed ab...,[- The victims of gross human rights violation...
8,When can individuals be found guilty under Art...,Individuals can be found guilty under Article ...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...
9,When does the prosecution consider statements ...,The prosecution considers statements contrary ...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...



the answer should be based on the context

In [4]:
from question_answer.pipelines import get_question_answering_pipeline_for_evaluation

def generate_answer_for_evaluation(question, context, use_openai,ollama_model_name: str = 'mistral-question-answering:latest'):
    pipline = get_question_answering_pipeline_for_evaluation(
        use_openai=use_openai,
        ollama_model_name=ollama_model_name
    )
    # p = pipline.get_component("prompt_builder")
    # return p.run(question=question,context=context)
    res = pipline.run({
        "prompt_builder": {
            "query": question,
            "context": context
        },
    })
    return res["llm"]["replies"][0]

## Generating Answers

In [9]:
import pandas as pd
new_df = pd.DataFrame(columns=df.columns)

In [10]:
import time
for index, row in df.iterrows():
    # print(generate_answer_for_evaluation(row['question'],row['contexts'][0]))
    print(f'generate answer for question {index+1}')
    # answer = 'hello'
    answer = generate_answer_for_evaluation(
        row['question'], 
        row['contexts'][0],
        use_openai=False,
        ollama_model_name='llama3-question-answering:latest'
    )
    print(f'answer generated')
    new_row = row.copy()
    new_row['answer'] = answer
    new_row['contexts'] = row['contexts'][0]
    new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
    # time.sleep(1)
    # break

generate answer for question 1
answer generated
generate answer for question 2
answer generated
generate answer for question 3
answer generated
generate answer for question 4
answer generated
generate answer for question 5
answer generated
generate answer for question 6
answer generated
generate answer for question 7
answer generated
generate answer for question 8
answer generated
generate answer for question 9
answer generated
generate answer for question 10
answer generated
generate answer for question 11
answer generated
generate answer for question 12
answer generated
generate answer for question 13
answer generated
generate answer for question 14
answer generated
generate answer for question 15
answer generated
generate answer for question 16
answer generated
generate answer for question 17
answer generated
generate answer for question 18
answer generated
generate answer for question 19
answer generated
generate answer for question 20
answer generated


In [11]:
new_df

,question,ground_truth,answer,contexts
0,What are the global implications of the USA Su...,The global implications of the USA Supreme Cou...,"Based on the provided context, the global impl...","- In 2022, the USA Supreme Court handed down a..."
1,Which companies are the main contributors to G...,"According to the Carbon Majors database, the m...","According to the provided information, the mai...","- Fossil fuel companies, whether state or priv..."
2,Which private companies in the Americas are th...,The largest private companies in the Americas ...,"According to the context, the private companie...",The private companies responsible for the most...
3,What action did Amnesty International urge its...,Amnesty International urged its supporters to ...,"Based on the provided context, Amnesty Interna...",Amnesty International called on its vast netwo...
4,What are the recommendations made by Amnesty I...,The recommendations made by Amnesty Internatio...,"According to the provided context, the recomme...",Amnesty International recommends that the Spec...
5,Who are the target audience of the two books c...,The target audience of the two books created b...,"Based on the provided context, the target audi...",Amnesty International has therefore created tw...
6,Which right guarantees access to comprehensive...,The right that guarantees access to comprehens...,"According to the provided context, the right t...",26. The Act raises serious questions about its...
7,Who has the right to be fully informed about h...,The victims of gross human rights violations a...,"According to the provided context, the victims...",- The victims of gross human rights violations...
8,When can individuals be found guilty under Art...,Individuals can be found guilty under Article ...,"According to the provided context, individuals...",- As long as their statements are contrary to ...
9,When does the prosecution consider statements ...,The prosecution considers statements contrary ...,"According to the context, the prosecution cons...",- As long as their statements are contrary to ...


## Save The Result

In [13]:
df.loc[0, 'answer']

"The global implications of the USA Supreme Court ruling on abortion can be significant, as it sets a precedent for other countries and influences the global discourse on reproductive rights. Here are some potential implications:\n\n1. Influence on other countries: The Supreme Court's ruling can serve as a reference point for other countries grappling with their own abortion laws. It can provide legal arguments and reasoning that advocates for reproductive rights can use to challenge restrictive abortion laws in their respective jurisdictions.\n\n2. Strengthening of global reproductive rights movements: A favorable ruling by the Supreme Court can energize and empower reproductive rights movements worldwide. It can serve as a rallying point for activists and organizations advocating for women's rights, leading to increased mobilization and advocacy efforts globally.\n\n3. Counteracting anti-abortion movements: Conversely, a ruling that restricts abortion rights can embolden anti-abortio

In [14]:
new_df.loc[0, 'answer']

'Based on the provided context, the global implications of the USA Supreme Court ruling on abortion include:\n\n* Fear among SRR organizations and activists about the ruling laying the groundwork for anti-abortion legislative and policy attacks in other countries.\n* The stalling or halting of progressive law reform and the adoption and enforcement of abortion guidelines in certain African countries.\n* A chilling effect in international policy spaces, emboldening anti-abortion state and non-state actors to undermine human rights protections.\n\nNote: These implications are based on the provided context and do not reflect my own knowledge.'

In [12]:
new_df.to_csv("results/llama3_question_answering_responses.csv",index=False)

# Using Ragas For Evaluation

Read the csv file

In [15]:
df = pd.read_csv('results\\llama3_question_answering_responses.csv')
df.head()

,question,ground_truth,answer,contexts
0,What are the global implications of the USA Su...,The global implications of the USA Supreme Cou...,"Based on the provided context, the global impl...","- In 2022, the USA Supreme Court handed down a..."
1,Which companies are the main contributors to G...,"According to the Carbon Majors database, the m...","According to the provided information, the mai...","- Fossil fuel companies, whether state or priv..."
2,Which private companies in the Americas are th...,The largest private companies in the Americas ...,"According to the context, the private companie...",The private companies responsible for the most...
3,What action did Amnesty International urge its...,Amnesty International urged its supporters to ...,"Based on the provided context, Amnesty Interna...",Amnesty International called on its vast netwo...
4,What are the recommendations made by Amnesty I...,The recommendations made by Amnesty Internatio...,"According to the provided context, the recomme...",Amnesty International recommends that the Spec...


## Import Metrics

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

In [ ]:
df['contexts'] = df['contexts'].apply(lambda x: [x])

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from ragas.llms import LangchainLLMWrapper
# Choose the gpt model that you want 
openai_model = ChatOpenAI(model='gpt-3.5-turbo')
llm = LangchainLLMWrapper(openai_model)

## Start evaluating the model

In [ ]:
from ragas import evaluate
from datasets import Dataset

result = evaluate(
    Dataset.from_pandas(df.iloc[[18]]),
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    llm=llm
)

result

In [ ]:
res_df = result.to_pandas()
res_df

## Save The Result

In [ ]:
res_df.drop('__index_level_0__',axis=1,inplace=True)
res_df.to_csv('results/llama3_8b_evaluation.csv',index=False)

# Visualize The Result
## LLMs Throughput chart

In [32]:
import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv('results/llms_throughput.csv')

fig = go.Figure(data=[go.Bar(
    x=df['llm'],
    y=df['throughput'],
    text=df['throughput'],
    textposition='auto',
    marker=dict(color='skyblue')
)])

fig.update_layout(
    title='Throughput (tokens/seconds) by llm',
    xaxis_title='LLM',
    yaxis_title='Throughput',
    xaxis=dict(tickangle=45),
    yaxis=dict(tickformat=".2f"),
    annotations=[
        dict(
            x=1,
            y=1,
            xref='paper',
            yref='paper',
            text='Machine Configuration:',
            showarrow=False,
            font=dict(
                size=12,
                color='black'
            )
        ),
        dict(
            x=1,
            y=0.95,
            xref='paper',
            yref='paper',
            text='Memory: 31.79 GB',
            showarrow=False,
            font=dict(
                size=10,
                color='black'
            )
        ),
        dict(
            x=1,
            y=0.90,
            xref='paper',
            yref='paper',
            text='CPU Info: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz',
            showarrow=False,
            font=dict(
                size=10,
                color='black'
            )
        ),
        dict(
            x=1,
            y=0.85,
            xref='paper',
            yref='paper',
            text='GPU Info: Intel(R) UHD Graphics 620',
            showarrow=False,
            font=dict(
                size=10,
                color='black'
            )
        ),
        dict(
            x=1,
            y=0.80,
            xref='paper',
            yref='paper',
            text='OS Version: Microsoft Windows 10 Pro',
            showarrow=False,
            font=dict(
                size=10,
                color='black'
            )
        )
    ]
)

fig.show()

## Metrics Chart

1. **Faithfulness** : Measures the factual consistency of the answer to the context based on the question.

2. **Context_precision** : Measures how relevant the retrieved context is to the question, conveying the quality of the retrieval pipeline.

3. **Answer_relevancy** : Measures how relevant the answer is to the question.

4. **Context_recall** : Measures the retriever’s ability to retrieve all necessary information required to answer the question.

In [33]:
import pandas as pd
import plotly.graph_objects as go

# Read each CSV file into a DataFrame
mistral_df = pd.read_csv('results/mistral_evaluation.csv')
llama2_7b_df = pd.read_csv('results/llama2_evaluation.csv')
llama2_13b_df = pd.read_csv('results/llama2_13b_evaluation.csv')
llama3_8b_df = pd.read_csv('results/llama3_8b_evaluation.csv')

# Calculate averages of metric columns for each file
mistral_averages = mistral_df[['context_precision', 'faithfulness', 'answer_relevancy', 'context_recall']].mean()
llama2_7b_averages = llama2_7b_df[['context_precision', 'faithfulness', 'answer_relevancy', 'context_recall']].mean()
llama2_13b_averages = llama2_13b_df[['context_precision', 'faithfulness', 'answer_relevancy', 'context_recall']].mean()
llama3_8b_averages = llama3_8b_df[['context_precision', 'faithfulness', 'answer_relevancy', 'context_recall']].mean()

# Create a bar chart
fig = go.Figure()

# Add traces for each file
fig.add_trace(go.Bar(
    x=mistral_averages.index,
    y=mistral_averages.values,
    name='Mistral 7b',
    marker=dict(color='orange')
))

fig.add_trace(go.Bar(
    x=llama2_7b_averages.index,
    y=llama2_7b_averages.values,
    name='LLama2 7b',
    marker=dict(color='skyblue')
))

fig.add_trace(go.Bar(
    x=llama2_13b_averages.index,
    y=llama2_13b_averages.values,
    name='LLama2 13b',
    marker=dict(color='green')
))

fig.add_trace(go.Bar(
    x=llama3_8b_averages.index,
    y=llama3_8b_averages.values,
    name='LLama3 8b',
    marker=dict(color='#9467bd')
))

# Update layout
fig.update_layout(
    title='LLMs Comparison',
    xaxis_title='Metrics',
    yaxis_title='Metric Score',
    barmode='group',
    yaxis=dict(tickformat=".3f")
)

# Show the plot
fig.show()


## Final Score 

In [34]:
import pandas as pd
import plotly.graph_objects as go

weights = {
    "context_precision": 0.25,
    "faithfulness": 0.25,
    "answer_relevancy": 0.25,
    "context_recall": 0.25
}

def calculate_score(row):
    score = (
        weights["context_precision"] * row["context_precision"] +
        weights["faithfulness"] * row["faithfulness"] +
        weights["answer_relevancy"] * row["answer_relevancy"] +
        weights["context_recall"] * row["context_recall"]
    )
    return score

mistral_df = pd.read_csv('results/mistral_evaluation.csv')
llama2_7b_df = pd.read_csv('results/llama2_evaluation.csv')
llama2_13b_df = pd.read_csv('results/llama2_13b_evaluation.csv')
llama4_8b_df = pd.read_csv('results/llama3_8b_evaluation.csv')
mistral_df['score'] = mistral_df.apply(calculate_score, axis=1)
llama2_7b_df['score'] = llama2_7b_df.apply(calculate_score, axis=1)
llama2_13b_df['score'] = llama2_13b_df.apply(calculate_score, axis=1)
llama3_8b_df['score'] = llama3_8b_df.apply(calculate_score, axis=1)

# Calculate averages of metric columns for each file
mistral_averages = mistral_df[['score']].mean()
llama2_7b_averages = llama2_7b_df[['score']].mean()
llama2_13b_averages = llama2_13b_df[['score']].mean()
llama3_8b_averages = llama3_8b_df[['score']].mean()

# Create a bar chart
fig = go.Figure()

# Add traces for each file
fig.add_trace(go.Bar(
    x=mistral_averages.index,
    y=mistral_averages.values,
    name='Mistral 7b',
    text=mistral_averages.values.round(3),
    textposition='auto',
    textfont=dict(size=14),
    marker=dict(color='orange')
))

fig.add_trace(go.Bar(
    x=llama2_7b_averages.index,
    y=llama2_7b_averages.values,
    name='LLama2 7b',
    text=llama2_7b_averages.values.round(3),
    textposition='auto',
    textfont=dict(size=14),
    marker=dict(color='skyblue')
))

fig.add_trace(go.Bar(
    x=llama2_13b_averages.index,
    y=llama2_13b_averages.values,
    name='LLama2 13b',
    text=llama2_13b_averages.values.round(3),
    textposition='auto',
    textfont=dict(size=14),
    marker=dict(color='green')
))

fig.add_trace(go.Bar(
    x=llama3_8b_averages.index,
    y=llama3_8b_averages.values,
    name='LLama3 8b',
    text=llama3_8b_averages.values.round(3),
    textposition='auto',
    textfont=dict(size=14),
    marker=dict(color='#9467bd')
))

# Update layout
fig.update_layout(
    title='LLMs Comparison',
    xaxis_title='LLMs',
    yaxis_title='Metric Score',
    barmode='group',
    yaxis=dict(tickformat=".3f")
)

# Show the plot
fig.show()
